In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.metrics import dtw_subsequence_path

# import xgboost as xgb

In [25]:
dataset_folder = "../all"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [26]:
def read_pickle(name):
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

In [27]:
xg_model = read_pickle("xg_model")
shapelets = read_pickle("shapelets")
print(shapelets.shape)

(36,)


/home/frankl1/.env/plasticc/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [28]:
classes = np.array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])
classes

array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])

In [29]:
def normalize(ts, ts_err):
    ts /= (ts_err + 1) # +1 to avoid zero division
    ts = np.nan_to_num(ts)
    ts = TimeSeriesScalerMinMax().fit_transform(ts)
    return ts

In [30]:
def distanceToShapelet(ts, shapelet):
    path, dist = dtw_subsequence_path(shapelet, ts)
    return dist

In [31]:
def distanceToShapelets(ts, shapelets=shapelets):
    return [distanceToShapelet(ts, shapelet) for shapelet in shapelets]

In [32]:
step = 1
nb_of_passband = 6
max_mjd = 1094
print(nb_of_passband, max_mjd, sep="\n")

6
1094


In [33]:
def format_time_series(data):
    X = []
    X_err = []
    s0 = int(data.shape[0] / nb_of_passband)
    x, x_err = [], []
    for p in range(nb_of_passband):
        x = np.append(x, data[data.passband==p].flux.values)
        x_err = np.append(x_err, data[data.passband==p].flux_err.values)
    X.append(x.reshape(s0, nb_of_passband))
    X_err.append(x.reshape(s0, nb_of_passband))
    return np.array(X), np.array(X_err)

In [34]:
def mjdToPredict(mjdToExclude, minMjd, maxMjd, step):
    return np.array(list(set(range(minMjd, maxMjd+1, step)) - set(mjdToExclude)))

In [35]:
def fill_missing_flux(objectDf, maxMjd, step):
    objectDf.mjd = objectDf[["mjd"]].astype(np.int64)
    objectDf.mjd = objectDf.mjd - objectDf.mjd.min()
    instance = objectDf.groupby(["passband", "mjd"]).mean().reset_index()
    meanByPassband = instance.groupby("passband").mean()[["flux", "flux_err"]]
    frames = [instance]
    for p in range(6):
        toExclude = instance[instance.passband==p].mjd
        toPredict = mjdToPredict(toExclude, 0, maxMjd, step)
        nb = len(toPredict)
        frames.append(pd.DataFrame({
            'mjd': toPredict,
            'flux': [meanByPassband.loc[p].flux]*nb,
            'flux_err': [meanByPassband.loc[p].flux_err]*nb,
            'detected': [-1] * nb,
            'passband': [p]*nb,
            'object_id': [objectDf.object_id.values[0]]*nb
        }))
    return pd.concat(frames, sort=False)

In [36]:
columns = ["object_id"]
columns.extend([f"class_{c}" for c in classes])
submission = pd.DataFrame(columns = columns)
submission

Empty DataFrame
Columns: [object_id, class_6, class_15, class_16, class_42, class_52, class_53, class_62, class_64, class_65, class_67, class_88, class_90, class_92, class_95, class_99]
Index: []

In [37]:
def predict_batch(test_df):
    instance = fill_missing_flux(test_df, max_mjd, step)
    ts, ts_err = format_time_series(instance)
    ts_norm = normalize(ts, ts_err)
    dist_vec = distanceToShapelets(ts_norm[0], shapelets)
    preds = xg_model.predict_proba([dist_vec])[0]
    preds = np.concatenate(([test_df.object_id.values[0]], preds, [0]))
    submission.loc[submission.shape[0]] = preds

In [38]:
max_nb_of_chunk = 10
start_at = 0
print("Starting at:", start_at)

Starting at: 0


In [43]:
def predict_test_set(start_at=0):
    test_example = []
    curr_id = None
    i = 0
    for chunk in pd.read_csv(f"{dataset_folder}/test_set.csv", chunksize=1000, skiprows=list(range(1, start_at+1))):
        i += 1
        if curr_id is None:
            curr_id = chunk.object_id.values[0]
        print(f"Batch {i}")
        test_example.append(chunk.copy())
        if chunk.object_id.values[-1] != curr_id:
            tail = pd.concat(test_example)
            obj_ids = tail.object_id.unique()
            while obj_ids.shape[0] > 1:
                one_object_df = tail[tail.object_id==obj_ids[0]]
                predict_batch(one_object_df)
                obj_ids = np.delete(obj_ids, 0)
                start_at += one_object_df.shape[0]
            test_example = [tail[tail.object_id.isin(obj_ids)]]
            current_id = None if tail.shape[0] == 0 else tail.object_id.values[0]    
#         if i != max_nb_of_chunk:
#             print("EOF")
#             break
    test_df = pd.concat(test_example)
    predict_batch(test_df)
    print("End at:", start_at)

In [ ]:
%%time
predict_test_set(start_at=start_at)

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106
Batch 107
Batch 108
Batch 109
Batch 110
Batch 11

In [ ]:
submission.object_id = submission.object_id.astype(np.int32, inplace=True)
submission.shape

In [ ]:
submission.to_csv("submission.csv", index=False)

In [13]:
((45365.3105 * 30) / (3600)) / 24

15.75184392361111